In [ ]:
from datasets import load_dataset
from datasets import Dataset
from transformers import GPT2LMHeadModel
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from huggingface_hub import login, upload_file
from transformers import AutoTokenizer
from datasets import DatasetDict
from huggingface_hub import Repository
import os
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments, Trainer
from transformers import TrainingArguments, Trainer, BertForSequenceClassification, BertTokenizer
import torch

In [ ]:
# Log in to Hugging Face


In [ ]:
# Load the CSV file
dataset = pd.read_csv("students_final (1).csv")

In [ ]:
print("Total Rows:", len(dataset))
print("Columns:", dataset.columns)
print("Sample Data:")
print(dataset.head())

Total Rows: 235
Columns: Index(['Age', 'Gender', 'field', 'Role', 'Exam preference',
       'Expression of understanding', 'MCQ : Limited scope',
       'MCQ : Easy to guess answers', 'MCQ : Misleading options',
       'MCQ : Time Constraints', 'Written : Time-consuming ',
       'Written : Requires more effort', 'Written : Subjective grading ',
       'Written :  Harder to revise', 'More Preparation time',
       'MCQ : memorization or understanding', 'MCQ : encouraging guessing ',
       'Stress', 'Chance for higher score', 'confidence MCQs vs. written',
       'MCQ : Grades', 'Written : Grades', 'Studying strategy',
       'MCQ : Suitablity', 'MCQ : Suitablity examples', 'MCQ : frequency',
       'Written : frequency', 'Evaluating critical thinking',
       'MCQ : Fairer grading', 'MCQ : Why fairer grading',
       'More time-efficient', 'Suggestions Y/N', 'Suggestions',
       'MCQ_Suitability_standardized', 'Chance for higher score_category',
       'MCQ : Why fairer grading_categ

In [ ]:
suggestions_data = dataset[['Suggestions']].dropna()
suggestions_data = suggestions_data[suggestions_data['Suggestions'] != 'No suggestions']

In [ ]:
suggestions_data = suggestions_data.drop_duplicates()

In [ ]:
# Check the cleaned dataset
print(f"Total Rows After Cleaning: {len(suggestions_data)}")
print(suggestions_data.head())

Total Rows After Cleaning: 46
                                          Suggestions
0   multiple-choice questions should be clear and ...
1                                       No suggestion
11  Offer more practice MCQs to boost confidence a...
13  Make written questions measure the level of un...
19  More time should be allocated for written asse...


In [ ]:
# Convert to Hugging Face Dataset format and rename column
hf_dataset = Dataset.from_pandas(suggestions_data.rename(columns={"Suggestions": "text"}))

In [ ]:
# Split into training and validation sets
train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
validation_dataset = train_test_split['test']

# Tokenize the Data

In [ ]:
# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Ensure compatibility with GPT-2
# Set the pad_token to the eos_token
tokenizer.pad_token = tokenizer.eos_token


# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=64, num_return_sequences=20) # Maximum sequence length


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Step 5: Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_valid = validation_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns (keeping tokenized data only)
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_valid = tokenized_valid.remove_columns(["text"])

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# Add labels by copying input_ids
# The labels are simply the input_ids because the model learns to predict the next token.
# Step 6: Add labels to the dataset (important for computing the loss during training)
def add_labels_to_input(example):
    example['labels'] = example['input_ids']
    return example

tokenized_train = tokenized_train.map(add_labels_to_input)
tokenized_valid = tokenized_valid.map(add_labels_to_input)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# Step 7: Set up the data collator for language modeling (since GPT-2 is a causal LM)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # GPT-2 is a causal language model, not a masked language model
)


# Fine-Tune GPT-2

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import wandb

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fayezrowan5 (fayezrowan5-collage) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [ ]:
from transformers import TrainingArguments, Trainer
# Enable fp16 only if on a CUDA-capable GPU
use_fp16 = torch.cuda.is_available()

# Define the training arguments (without max_length)
training_args = TrainingArguments(
    output_dir="./results",           # Directory where model predictions and checkpoints will be stored
    learning_rate=0.0001,  # Learning rate for fine-tuning
    num_train_epochs=1,               # Reduced epochs
    per_device_train_batch_size=2,    # Further reduced batch size
    per_device_eval_batch_size=2,     # Further reduced batch size
    warmup_steps=500,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir="./logs",             # Directory for storing logs
    logging_steps=100,                # Log less frequently
    do_train=True,                    # Enable training
    do_eval=True,                     # Enable evaluation
    evaluation_strategy="epoch",      # Evaluate every epoch
    save_strategy="epoch",           # Save model every epoch
    save_total_limit=2,               # Keep only the last 2 checkpoints
    load_best_model_at_end=True,      # Load the best model at the end of training
    metric_for_best_model="loss",     # Metric used to determine the best model
    gradient_accumulation_steps=16,   # Gradient accumulation
    report_to=["wandb"],  # Log metrics to Weights & Biases
    fp16=use_fp16,                    # Conditionally enable fp16 based on GPU availability
)

# Enable gradient checkpointing
model.config.gradient_checkpointing = True  # This saves memory during training


# Define the trainer
trainer = Trainer(
    model=model,                        # The pre-trained model
    args=training_args,                 # Training arguments
    train_dataset=tokenized_train,      # The training dataset
    eval_dataset=tokenized_valid,       # The validation dataset
    tokenizer=tokenizer,                # Tokenizer for preprocessing
)


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:645: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [ ]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Epoch,Training Loss,Validation Loss
0,No log,11.151117


TrainOutput(global_step=1, training_loss=9.751897811889648, metrics={'train_runtime': 13.5059, 'train_samples_per_second': 2.665, 'train_steps_per_second': 0.074, 'total_flos': 16722690048000.0, 'train_loss': 9.751897811889648, 'epoch': 0.89})

In [ ]:
# Save the fine-tuned model
trainer.save_model("./fine_tuned_gpt2_suggestions")

In [ ]:
# Load the saved fine-tuned model and tokenizer
model_path = "./fine_tuned_gpt2_suggestions"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [ ]:
# Set the model to evaluation mode
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# Function to generate suggestions
def generate_suggestion(prompt, max_length=50):
    # Encode the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate output
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    # Decode and return the generated text
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Example usage
prompt = "Students prefer multiple-choice exams because"
suggestion = generate_suggestion(prompt)
print("Generated Suggestion:")
print(suggestion)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Suggestion:
Students prefer multiple-choice exams because of their preference for the more standard-based, standardized exams.

In addition to the general rule of thumb, students who don't like the traditional test include students with learning disabilities, who are more likely to


In [ ]:
prompt2 = "Written exams are challenging due to"
suggestion2 = generate_suggestion(prompt2)
print("\nGenerated Suggestion 2:")
print(suggestion2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Suggestion 2:
Written exams are challenging due to the number of times they need to be completed before the final exam.

The only way to ensure that you're able to achieve your dream is to have a full exam, and to keep your exams as long as
